## Data Ingestion

In [1]:
import pandas as pd

# column_names = ['id', 'text', 'label']
twitter_train = pd.read_csv("/kaggle/input/dataset-3/train_formatted.csv")
twitter_test = pd.read_csv("/kaggle/input/dataset-3/test_formatted.csv")
twitter_dev = pd.read_csv("/kaggle/input/dataset-3/dev_formatted.csv")

In [2]:
twitter_train.shape, twitter_test.shape, twitter_dev.shape

((1557, 5), (520, 5), (485, 5))

## Data Preparation

In [3]:
LABELS = twitter_train['label'].unique().tolist()
train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3616623573.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [4]:
train_data = twitter_train['text'].tolist()

In [5]:
dev_data = twitter_dev['text'].tolist()
dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3987848631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [6]:
test_data = twitter_test['text'].tolist()
test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/2548705681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [8]:
from transformers import BartTokenizer, BartModel, AdamW

max_length = 128

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [10]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [11]:
num_classes = 4

In [12]:
class BARTClassifier(nn.Module):
    def __init__(self, bart_model, num_classes, dropout=0.1):
        super(BARTClassifier, self).__init__()
        self.bart = bart_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bart(input_ids=input_ids, attention_mask=attention_mask)
        
        last_hidden_state = outputs.last_hidden_state  
        
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_hidden_states = torch.sum(last_hidden_state * attention_mask_expanded, 1)
        sum_attention_mask = torch.clamp(attention_mask_expanded.sum(1), min=1e-9)
        pooled_output = sum_hidden_states / sum_attention_mask
        
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [13]:
bart_model = BartModel.from_pretrained('facebook/bart-base')

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [14]:
model = BARTClassifier(bart_model, num_classes)

In [15]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BARTClassifier(
  (bart): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Layer

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [17]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [18]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [19]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 49/49 [00:22<00:00,  2.14it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 1/5
Training Metrics:
Loss: 1.3987
Accuracy: 0.2749
Precision: 0.2759
Recall: 0.2749
F1: 0.2751
Roc_auc: 0.5181

Validation Metrics:
Accuracy: 0.2969
Precision: 0.3768
Recall: 0.2969
F1: 0.1968
Roc_auc: 0.5521


Epoch 2/5: 100%|██████████| 49/49 [00:21<00:00,  2.25it/s]



Epoch 2/5
Training Metrics:
Loss: 1.3728
Accuracy: 0.2942
Precision: 0.2981
Recall: 0.2942
F1: 0.2920
Roc_auc: 0.5599

Validation Metrics:
Accuracy: 0.3052
Precision: 0.3309
Recall: 0.3052
F1: 0.3007
Roc_auc: 0.6071


Epoch 3/5: 100%|██████████| 49/49 [00:21<00:00,  2.24it/s]



Epoch 3/5
Training Metrics:
Loss: 1.2955
Accuracy: 0.3738
Precision: 0.3748
Recall: 0.3738
F1: 0.3686
Roc_auc: 0.6509

Validation Metrics:
Accuracy: 0.3588
Precision: 0.5245
Recall: 0.3588
F1: 0.3018
Roc_auc: 0.6758


Epoch 4/5: 100%|██████████| 49/49 [00:21<00:00,  2.24it/s]



Epoch 4/5
Training Metrics:
Loss: 1.1699
Accuracy: 0.4631
Precision: 0.4786
Recall: 0.4631
F1: 0.4634
Roc_auc: 0.7314

Validation Metrics:
Accuracy: 0.4165
Precision: 0.5230
Recall: 0.4165
F1: 0.3875
Roc_auc: 0.6956


Epoch 5/5: 100%|██████████| 49/49 [00:21<00:00,  2.23it/s]



Epoch 5/5
Training Metrics:
Loss: 1.0138
Accuracy: 0.5478
Precision: 0.5499
Recall: 0.5478
F1: 0.5479
Roc_auc: 0.8101

Validation Metrics:
Accuracy: 0.4680
Precision: 0.5381
Recall: 0.4680
F1: 0.4500
Roc_auc: 0.7600


In [21]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,1.398662,0.274888,0.275871,0.274888,0.275122,0.518078
1,1.372799,0.294155,0.298124,0.294155,0.292012,0.559872
2,1.295494,0.373796,0.374798,0.373796,0.368625,0.650887
3,1.169875,0.463070,0.478559,0.463070,0.463365,0.731411
4,1.013805,0.547848,0.549853,0.547848,0.547880,0.810071


In [22]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.296907,0.376760,0.296907,0.196826,0.552061
1,0.305155,0.330851,0.305155,0.300745,0.607122
2,0.358763,0.524548,0.358763,0.301805,0.675825
3,0.416495,0.522953,0.416495,0.387463,0.695595
4,0.468041,0.538070,0.468041,0.450003,0.759994


In [23]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.4480769230769231,
 'precision': 0.488505638139971,
 'recall': 0.4480769230769231,
 'f1': 0.43469232877811625,
 'roc_auc': 0.7397927481391136}